In [1]:
import pandas as pd
import pymongo
import re 
import random
from copy import deepcopy
import requests, json

In [30]:
icd2sct = pd.read_table('Download/Mapping/mapSnomedToIcd.txt')

In [31]:
icd2sct

,conceptId,亚目编码,mapAdvice,term,亚目名称
0,240349003,A00.0,ALWAYS A00.0,Cholera due to Vibrio cholerae O1 Classical bi...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
1,240349003,A00.0,ALWAYS A00.0,Cholera caused by Vibrio cholerae O1 Classical...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
2,240349003,A00.0,ALWAYS A00.0,Cholera caused by Vibrio cholerae O1 Classical...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
3,81020007,A00.1,ALWAYS A00.1,Cholera due to Vibrio cholerae El Tor,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
4,81020007,A00.1,ALWAYS A00.1,Cholera - Vibrio cholerae O1 El Tor biotype,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
5,81020007,A00.1,ALWAYS A00.1,Cholera caused by Vibrio cholerae El Tor,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
6,81020007,A00.1,ALWAYS A00.1,Cholera caused by Vibrio cholerae El Tor (diso...,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
7,63650001,A00.9,ALWAYS A00.9,Cholera,未特指的霍乱
8,63650001,A00.9,ALWAYS A00.9,Cholera (disorder),未特指的霍乱
9,63650001,A00.9,ALWAYS A00.9,Vibrio cholerae infection,未特指的霍乱


In [5]:
for line in list(icd2sct['conceptId']):
    assert type(line) == int

In [6]:
rela = pd.read_table('Download/SnomedCT_InternationalRF2_PRODUCTION_20190731T120000Z/Delta/Terminology/sct2_Relationship_Delta_INT_20190731.txt')

In [7]:
rela = rela[rela['active']==1]

In [8]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client.sym_info.sym_info

In [9]:
sym_sctid_list = [re.findall(r'sctid-(\d+)', item['bmesh'])[0] for item in db.find() if len(re.findall(r'sctid-(\d+)', item['bmesh']))>0]

In [10]:
set(icd2sct['conceptId']).intersection(set(sym_sctid_list))

set()

In [32]:
icd_id_set = set(icd2sct['conceptId'])
sym_id_set = set(sym_sctid_list)

NameError: name 'sym_sctid_list' is not defined

In [12]:
descri = pd.read_table('Download/SnomedCT_InternationalRF2_PRODUCTION_20190731T120000Z/Delta/Terminology/sct2_Description_Delta-en_INT_20190731.txt')

In [13]:
descri = descri[descri['active']==1]

In [16]:
for row in rela.iterrows():
    sctids = tuple(row[1][['sourceId', 'destinationId']])
    if sctids[0] in icd_id_set and len(descri[descri['conceptId']==sctids[1]]) != 0:
        print('sctids[0] in icd_id_set')
        print(icd2sct[icd2sct['conceptId']==sctids[0]])
        print('-----------------------------')
        print(descri[descri['conceptId']==sctids[1]])
        print(' *****************************\n *****************************\n *****************************\n')
        break
    elif sctids[1] in icd_id_set and len(descri[descri['conceptId']==sctids[0]]) != 0:
        print('sctids[1] in icd_id_set')
        print(icd2sct[icd2sct['conceptId']==sctids[1]])
        print('-----------------------------')
        print(descri[descri['conceptId']==sctids[0]])
        print(' *****************************\n *****************************\n *****************************\n')

sctids[1] in icd_id_set
        conceptId   亚目编码                                          mapAdvice  \
195302  129135003  T11.3  ALWAYS T11.3 | POSSIBLE REQUIREMENT FOR AN EXT...   
195303  129135003  T11.3  ALWAYS T11.3 | POSSIBLE REQUIREMENT FOR AN EXT...   

                                                 term              亚目名称  
195302             Injury of nerve of upper extremity  上肢未特指神经的损伤,水平未特指  
195303  Injury of nerve of upper extremity (disorder)  上肢未特指神经的损伤,水平未特指  
-----------------------------
               id  effectiveTime  active            moduleId  conceptId  \
12385  3767137017       20190731       1  900000000000207008  212295006   
12386  3767138010       20190731       1  900000000000207008  212295006   

      languageCode              typeId  \
12385           en  900000000000003001   
12386           en  900000000000013009   

                                                term  caseSignificanceId  
12385  Injury of intercostobrachial nerve (disorder)  9000

In [42]:
data = []
for row in rela.iterrows():
    row_result = {}
    sctids = tuple(row[1][['sourceId', 'destinationId']])
    if sctids[0] in icd_id_set and len(descri[descri['conceptId']==sctids[1]]) != 0:
        icd_result = icd2sct[icd2sct['conceptId']==sctids[0]]
        row_result['icd_id'] = list(icd_result['亚目编码'])[0]
        row_result['icd_name'] = list(icd_result['亚目名称'])[0]
        row_result['icd_where'] = 'source'
        row_result['another_sctid'] = deepcopy(sctids[1])
        descri_result = descri[descri['conceptId']==sctids[1]]
        row_result['another_sct_term'] = list(descri_result['term'])[0]
        row_result['relation_type_id'] = row[1]['typeId']
        data.append(deepcopy(row_result))
    
    elif sctids[1] in icd_id_set and len(descri[descri['conceptId']==sctids[0]]) != 0:
        icd_result = icd2sct[icd2sct['conceptId']==sctids[1]]
        row_result['icd_id'] = list(icd_result['亚目编码'])[0]
        row_result['icd_name'] = list(icd_result['亚目名称'])[0]
        row_result['icd_where'] = 'destination'
        row_result['another_sctid'] = deepcopy(sctids[0])
        descri_result = descri[descri['conceptId']==sctids[0]]
        row_result['another_sct_term'] = list(descri_result['term'])[0]
        row_result['relation_type_id'] = row[1]['typeId']
        data.append(deepcopy(row_result))
        

In [46]:
data_df = pd.DataFrame(data)[['icd_id', 'icd_name', 'icd_where', 'another_sctid', 'another_sct_term', 'relation_type_id', ]]

#### 翻译 sct term

In [60]:
def caiyunQuery(q):
    url = "http://api.interpreter.caiyunai.com/v1/translator"
 
    token = "mql8jp2ciiq0mqjqbrmo"

    payload = {
            "source" : q, 
            "trans_type" : "en2zh",
            "request_id" : "demo",
            }

    headers = {
            'content-type': "application/json",
            'x-authorization': "token " + token,
    }

    response = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    #print(response.text[:2500])

    return json.loads(response.text)

In [61]:
caiyunResult = []
for i in range(4604//50+1):
    temp = list(data_df['another_sct_term'])
    res = caiyunQuery(temp[i*50:(i+1)*50])
    caiyunResult += res['target']

In [63]:
assert len(caiyunResult) == 4605 

In [64]:
data_df['another_sct_term_cn'] = caiyunResult

In [65]:
data_df

,icd_id,icd_name,icd_where,another_sctid,another_sct_term,relation_type_id,another_sct_term_cn
0,T11.3,"上肢未特指神经的损伤,水平未特指",destination,212295006,Injury of intercostobrachial nerve (disorder),116680003,肋间神经损伤(失调)
1,R04.0,鼻出血,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱
2,Q31.9,喉未特指的先天性畸形,destination,232461002,Congenital fissure of larynx,116680003,先天性喉裂
3,K26.9,"十二指肠溃疡未特指为急性或慢性,不伴有出血或穿孔",source,196652006,Acute peptic ulcer of duodenum,116680003,急性十二指肠消化性溃疡
4,L98.9,皮肤和皮下组织未特指的疾患,destination,238561006,Chronic vesicular eczema of foot,116680003,足部慢性水疱性湿疹
5,G71.0,肌营养不良,source,240046001,Limb-girdle muscular dystrophy,116680003,肢节型肌营养不良症
6,S09.9,头部未特指的损伤,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱
7,T94.1,"损伤后遗症,未按身体部位特指者",destination,285236005,Disorder due to and following injury of upper ...,116680003,上肢损伤(障碍)所致或继发的功能障碍
8,D10.1,舌良性肿瘤,source,91975001,Benign neoplasm of body of tongue,116680003,舌体良性肿瘤
9,D38.5,其他呼吸器官动态未定或动态未知的肿瘤,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱


#### 查询 relation type

In [66]:
with open('snomed_relationship_type.txt') as fget:
    lines = [line.strip() for line in fget.readlines()]

In [82]:
relationship_type = [re.search(pattern=r'(\d*)\ \|(.*)\|', string=line).groups() for line in lines]

In [84]:
relationship_type = {pair[0]: pair[1] for pair in relationship_type}

In [89]:
def look_type(l, d=relationship_type):
    return [d[str(item)] for item in l]

In [90]:
data_df['relationship_type'] = look_type(data_df['relation_type_id'])

In [93]:
output = data_df[['icd_id','icd_name','another_sctid','another_sct_term_cn','another_sct_term','relationship_type','icd_where',]]

In [95]:
output.to_csv('Snomed CT 中疾病相关关系（仅包含有term可查的）.tsv', index=None, sep='\t')

In [8]:
df2 = pd.read_table('Snomed_CT_relation.tsv')

In [10]:
df2.sort_values(by='relationship_type')

,icd_id,icd_name,another_sctid,another_sct_term_cn,another_sct_term,relationship_type,icd_where
494,B02.3,带状疱疹眼病,38907003,水痘,Chickenpox,After,source
258,S04.9,未特指的脑神经损伤,36539009,颅神经损伤及其后遗症(疾病),Disorder due to and following injury of crania...,After,destination
261,T65.9,未特指物质的毒性效应,363304009,中毒的后遗症,Late effect of poisoning,After,destination
4447,S44.9,在肩和上臂水平的未特指神经的损伤,784382001,肩部神经损伤的晚期效应,Late effect of injury of nerve of shoulder,After,destination
4450,T11.3,"上肢未特指神经的损伤,水平未特指",784383006,上肢神经损伤及其后遗症,Disorder due to and following injury of nerve ...,After,destination
268,B94.8,其他特指传染病和寄生虫病的后遗症,38907003,水痘,Chickenpox,After,source
4459,T85.7,其他内部假体装置、植入物和移植物引起的感染和炎症性反应,782902008,植入程序(程序),Implantation procedure (procedure),After,source
252,T84.6,内部固定装置[任何部位]引起的感染和炎症性反应,118470002,应用内固定系统进行内固定,Internal fixation using internal fixator system,After,source
4484,T65.9,未特指物质的毒性效应,218249005,意外中毒后遗症,Sequela of accidental poisoning,After,destination
610,A09.0,其他和未特指传染性病因的胃肠炎和结肠炎,239785008,痢疾后反应性关节炎,Post-dysenteric reactive arthritis,After,destination


In [14]:
relationship_types = list(df2['relationship_type'])

In [28]:
counts = [{'ty': ty, 'count': relationship_types.count(ty)} for ty in set(relationship_types)]

In [29]:
pd.DataFrame(counts)[['ty', 'count']]

,ty,count
0,Interprets,114
1,Associated with,14
2,Associated morphology,309
3,Associated procedure,14
4,Has focus,13
5,Causative agent,87
6,Associated finding,51
7,Due to,214
8,After,76
9,Is a (attribute),3047


In [36]:
df_disease = df2[df2.isin({'another_sctid': set(icd2sct['conceptId'])})['another_sctid']]

In [40]:
df_disease_counts = [{'ty': ty, 'count': list(df_disease['relationship_type']).count(ty)} for ty in set(df_disease['relationship_type'])]
pd.DataFrame(df_disease_counts)[['ty', 'count']]

,ty,count
0,Associated with,3
1,Due to,138
2,After,52
3,Is a (attribute),2736
4,Associated finding,40


In [41]:
df_disease[df_disease['relationship_type'] == 'Due to']

,icd_id,icd_name,another_sctid,another_sct_term_cn,another_sct_term,relationship_type,icd_where
241,E14.9,未特指的糖尿病不伴有并发症,201251005,糖尿病所致的足部神经病理性溃疡,Neuropathic ulcer of foot due to diabetes mell...,Due to,destination
244,E14.9,未特指的糖尿病不伴有并发症,236499007,糖尿病性微量白蛋白尿肾病,Microalbuminuric nephropathy due to diabetes m...,Due to,destination
245,E14.9,未特指的糖尿病不伴有并发症,236500003,糖尿病蛋白尿肾病,Proteinuric nephropathy due to diabetes mellit...,Due to,destination
246,E14.9,未特指的糖尿病不伴有并发症,238982009,糖尿病引起的皮肤病,Dermopathy due to diabetes mellitus (disorder),Due to,destination
250,E14.9,未特指的糖尿病不伴有并发症,309426007,糖尿病性肾小球疾病,Glomerulopathy due to diabetes mellitus (disor...,Due to,destination
251,E14.9,未特指的糖尿病不伴有并发症,310387003,糖尿病性肾小球硬化,Intracapillary glomerulosclerosis due to diabe...,Due to,destination
253,E14.9,未特指的糖尿病不伴有并发症,312912001,糖尿病引起的黄斑水肿,Macular edema due to diabetes mellitus (disorder),Due to,destination
254,E70.2,酪氨酸代谢紊乱,360378009,遗传性卵巢病,Hereditary ochronosis,Due to,source
255,E14.9,未特指的糖尿病不伴有并发症,127014009,糖尿病性周围血管病变,Peripheral angiopathy due to diabetes mellitus...,Due to,destination
256,E70.2,酪氨酸代谢紊乱,360378009,遗传性卵巢病,Hereditary ochronosis,Due to,source


In [42]:
df_disease[df_disease['relationship_type'] == 'After']

,icd_id,icd_name,another_sctid,another_sct_term_cn,another_sct_term,relationship_type,icd_where
242,T11.9,"上肢未特指的损伤,水平未特指",210960009,上肢肌肉、肌腱损伤后遗症(疾病),Disorder due to and following injury of muscle...,After,destination
243,T13.9,"下肢未特指的损伤,水平未特指",210965004,下肢肌腱损伤后遗症,Sequela of injury of muscle and tendon of lowe...,After,destination
247,T14.4,身体未特指部位的神经损伤,111729006,神经损伤(疾病)引起的或随后引起的疾病,Disorder due to and following injury to nerve ...,After,destination
249,T11.9,"上肢未特指的损伤,水平未特指",285236005,上肢损伤(障碍)所致或继发的功能障碍,Disorder due to and following injury of upper ...,After,destination
258,S04.9,未特指的脑神经损伤,36539009,颅神经损伤及其后遗症(疾病),Disorder due to and following injury of crania...,After,destination
268,B94.8,其他特指传染病和寄生虫病的后遗症,38907003,水痘,Chickenpox,After,source
328,B02.3,带状疱疹眼病,38907003,水痘,Chickenpox,After,source
329,G50.8,三叉神经的其他疾患,38907003,水痘,Chickenpox,After,source
330,B00.2,疱疹病毒性龈口炎和咽扁桃体炎,38907003,水痘,Chickenpox,After,source
331,B02.8,带状疱疹伴有其他并发症,38907003,水痘,Chickenpox,After,source


In [43]:
df_disease[df_disease['relationship_type'] == 'Associated finding']

,icd_id,icd_name,another_sctid,another_sct_term_cn,another_sct_term,relationship_type,icd_where
729,H50.9,未特指的斜视,16535691000119106,斜视史(情况),History of strabismus (situation),Associated finding,destination
748,D69.6,未特指的血小板减少,49341000119108,低血小板计数的历史,History of thrombocytopenia,Associated finding,destination
1673,O35.0,为胎儿(可疑)中枢神经系统畸形给予的孕产妇医疗\t,783724009,胎儿脊柱裂(疾病),Fetal spina bifida (disorder),Associated finding,source
1678,O35.0,为胎儿(可疑)中枢神经系统畸形给予的孕产妇医疗\t,783726006,胎儿脑积水,Foetal hydrocephalus,Associated finding,source
2306,Z86.0,其他肿瘤个人史,189948006,神经鞘瘤,Neurilemmoma,Associated finding,source
2514,H47.0,"视神经疾患,不可归类在他处者",16093711000119100,前部缺血性视神经病变史,History of anterior ischaemic optic neuropathy,Associated finding,destination
2561,Z84.8,其他特指情况家族史,267340006,产时母体发热,Maternal pyrexia during labour,Associated finding,source
2574,H43.8,玻璃体的其他疾患,16615741000119105,玻璃体后脱离病史(情况),History of posterior vitreous detachment (situ...,Associated finding,destination
3512,I67.1,"脑动脉瘤,未破裂",16845421000119109,脑内的动脉瘤的历史,History of cerebral aneurysm,Associated finding,destination
3514,I62.9,未特指的颅内出血(非创伤性),16864911000119103,颅内出血的历史,History of intracranial hemorrhage,Associated finding,destination


In [63]:
#rela_result = []
for row in rela.iterrows():
    sctids = tuple(row[1][['sourceId', 'destinationId']])
    if (sctids[0] in icd_id_set and sctids[1] in sym_id_set) or (sctids[1] in icd_id_set and sctids[0] in sym_id_set):
        print(sctids)
#    res = [0, 0]
#    for sctid in sctids:
#        if sctid in icd_id_set:
#            res[0] = 1
#            print(sctids, sctid)
#        elif sctid in sym_id_set:
#            res[1] = 1
#            print(sctids, sctid)
#    if sum(res) == 2:
#        rela_result.append(row)

In [52]:
tuple(row[1][['sourceId', 'destinationId']])

(100000000, 102272007)

In [59]:
rela_result

[]